## Stanford drone dataset to keras retinanet format
* Dataset: http://cvgl.stanford.edu/projects/uav_data/

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import time

In [ ]:
rename_path = lambda x, frame_path: os.path.join(frame_path, 'img_' + str(x).zfill(5) + '.jpg')

In [ ]:
def label_to_mapping_dict(label_df):
    sorted_labels = sorted(label_df.unique())
    return dict(zip(sorted_labels, range(len(sorted_labels))))

In [ ]:
def anno_writer(in_path, out_path, frame_path, filter_label_list=[]):
    data = pd.read_csv(in_path, header=None, sep='\s+')
    data.drop([0, 6, 7, 8], axis=1, inplace=True)
    data.columns = ['x1', 'y1', 'x2', 'y2', 'frame', 'label']
    data = data[['frame','x1', 'y1', 'x2', 'y2', 'label']]    
    data.sort_values(by='frame', inplace=True)
    data.reset_index(inplace=True, drop=True)
    label_mapping = label_to_mapping_dict(data['label'])
    data[~data['label'].isin(filter_label_list)]
    data['frame'] = data['frame'].apply(lambda x: rename_path(x, frame_path))
    data.replace({'label':label_mapping}, inplace=True)
    abs_file_path = os.path.join(out_path, 'annotations.csv')
    data.to_csv(abs_file_path, encoding='utf-8', index=False, header=False)

In [ ]:
def im_writer(video_path, out_path):
    count = 0
    cap = cv2.VideoCapture(video_path)

    while True:
        try:
            ret, frame = cap.read()
            if not ret:
                break
            img_name = 'img_'+str(count).zfill(5)+'.jpg'
            full_path = os.path.join(out_path, img_name)
            cv2.imwrite(full_path, frame)
            count += 1
        except Exception as e:
            print(e)

    print('Completed...!')
    if cap is not None:
        cap.release()
    print('Pausing for 10 seconds...')
    time.sleep(10)
    print('Resuming operation....')

In [ ]:
def get_file_path(in_path):
    anno_path = []
    video_path = []
    abs_out_frame_path = []
    abs_out_anno_path = []
    for root, dirs, files in os.walk(in_path):
        for file_name in files:
            abs_file_path = os.path.join(root, file_name)
            abs_out_path = os.path.join(out_path, os.path.relpath(root, in_path))
            if file_name.endswith('txt'):
                anno_path.append(abs_file_path)
                abs_out_anno_path.append(abs_out_path)
            elif file_name.endswith('mov'):
                video_path.append(abs_file_path)
                abs_out_frame_path.append(abs_out_path)
    
    return anno_path, video_path, abs_out_anno_path, abs_out_frame_path

In [ ]:
def sdd_to_retinanet(in_path, out_path, filter_label_list):
    anno_path, video_path, abs_out_anno_path, abs_out_frame_path = get_file_path(in_path)
    
    for i in range(len(anno_path)):
        if not os.path.isdir(abs_out_frame_path[i]):
            os.makedirs(abs_out_frame_path[i])
        video_file_path = video_path[i]
        print('Working on video: ', video_file_path)
        im_writer(video_file_path, abs_out_frame_path[i])
        
        if not os.path.isdir(abs_out_anno_path[i]):
            os.makedirs(abs_out_anno_path[i])
        anno_file_path = anno_path[i]
        print('Working on annotation: ', anno_file_path)
        anno_writer(anno_file_path, abs_out_anno_path[i], abs_out_frame_path[i], filter_label_list)

In [ ]:
in_path = '/home/anuj/dataset/abg/drone/stanford_campus_dataset'
out_path = '/home/anuj/dataset/abg/drone/stanford_campus_dataset_retinanet'
filter_label_list = ['Skater']

In [ ]:
sdd_to_retinanet(in_path, out_path, filter_label_list)

# References
* https://stackoverflow.com/questions/40828450/how-to-copy-folder-structure-under-another-directory